# MODELING

Since predicting Airbnb listing price is a continuous outcome, I will use regression models. First I will create a linear regression model predicting price with all of the predictor variables and use this as my baseline model to compare other models against. The other models I will use are ridge regression, lasso regression, random forest regression, support vector regression and gradient boosting.

I will evaluate and compare the performace of the models using two different metrics, root mean squared error (RMSE) and r-squared. RMSE measures the magnitude of the residuals or margin of error. R-squared measures the proportion of variance for the target variable that is explained by the independent variables. Ideally, lower RMSE (closer to zero) and higher r-squared (closer to 1) values are indicative of a better model.

Before diving into modeling, I need to complete some cleanup steps. 

# DATA PREPARATION

In [3]:
# Load the required libraries and modules
%matplotlib inline 

import timeit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cufflinks as cf
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
cf.go_offline()

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Read in the CSV file
df = pd.read_csv('Data/airbnb_clean.csv')
df.head()

,listing_id,zip_code,latitude,longitude,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,number_of_reviews,review_scores_rating,neighbourhood,number_of_bookings,bedroom_bath_ratio
0,2265,78702,30.27750,-97.71398,Entire home/apt,4,2.0,2.0,2.0,225.0,30,24,93.0,East Downtown,365.0,100.0
1,5245,78702,30.27577,-97.71379,Private room,2,1.0,1.0,2.0,100.0,30,9,91.0,East Downtown,354.0,100.0
2,5456,78702,30.26112,-97.73448,Entire home/apt,3,1.0,1.0,2.0,95.0,2,499,96.0,East Downtown,74.0,100.0
3,75174,78702,30.24773,-97.72584,Entire home/apt,3,1.0,1.0,1.0,130.0,2,249,98.0,East Downtown,131.0,100.0
4,76911,78702,30.26775,-97.72695,Entire home/apt,10,3.0,5.0,12.0,821.0,2,126,99.0,East Downtown,56.0,60.0


To ease the prediction of price, the distribution should be normally distributed. From prior exploratory data analysis and statistical inference, I observed that the data was heavily skewed to the right which means it is not normally distributed. I will transform price to log10.

In [5]:
# Convert raw price column to log
df['price'] = np.log10(df.price)

I will drop some columns that I don't think I need in the modeling. These columns are the listing id, zip code, latitude and longitude.

In [6]:
# Drop columns
drop_cols = ['listing_id', 'zip_code', 'latitude', 'longitude']
df.drop(columns=drop_cols, axis=1, inplace=True)

In [7]:
# Inspect new df
df.head()

,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,number_of_reviews,review_scores_rating,neighbourhood,number_of_bookings,bedroom_bath_ratio
0,Entire home/apt,4,2.0,2.0,2.0,2.352183,30,24,93.0,East Downtown,365.0,100.0
1,Private room,2,1.0,1.0,2.0,2.000000,30,9,91.0,East Downtown,354.0,100.0
2,Entire home/apt,3,1.0,1.0,2.0,1.977724,2,499,96.0,East Downtown,74.0,100.0
3,Entire home/apt,3,1.0,1.0,1.0,2.113943,2,249,98.0,East Downtown,131.0,100.0
4,Entire home/apt,10,3.0,5.0,12.0,2.914343,2,126,99.0,East Downtown,56.0,60.0


The regression models I will be using can not handle categorical variables unless I convert them to numerical values. There are two categorical variables in the data, room_type and neighbourhood. I will use the one hot encoding method to map each category to a vector that contains 1 and 0, denoting the presence or absence of that variable. 

In [8]:
# Perform get_dummies to encode categorical variables
df = pd.get_dummies(df, columns=['room_type', 'neighbourhood'], drop_first=True)

# Confirm that only numeric variables remain
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11333 entries, 0 to 11332
Data columns (total 44 columns):
accommodates                                11333 non-null int64
bathrooms                                   11333 non-null float64
bedrooms                                    11333 non-null float64
beds                                        11333 non-null float64
price                                       11333 non-null float64
minimum_nights                              11333 non-null int64
number_of_reviews                           11333 non-null int64
review_scores_rating                        11333 non-null float64
number_of_bookings                          11333 non-null float64
bedroom_bath_ratio                          11333 non-null float64
room_type_Hotel room                        11333 non-null uint8
room_type_Private room                      11333 non-null uint8
room_type_Shared room                       11333 non-null uint8
neighbourhood_Balcones Civic Ass

My next step is to create arrays of the dependent target (y) and independent predictor (X) variables.

In [9]:
# Split data into target and predictors 
y = df.price.values
X = df.drop('price', axis = 1).values

The numeric variables need to be scaled for more uniform and fair influence for all weights.

In [10]:
# Transform the depdendent variables
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

I will initialize an array for the MSE and r-squared scores because I will be using both for evaluation throughout this notebook. MSE will be converted to RMSE for comparison. 

In [11]:
# Array for mean squared error and r-squared 
scoring = ['neg_mean_squared_error', 'r2']

# METHODS AND RESULTS

<b>LINEAR REGRESSION</b>
<p>
I will run linear regression and use this as my baseline model to compare the other ones against. I will shuffle the data first to ensure that it's random. This model will use all of the available predictor variables. For better confidence in my prediction accuracy, I will use 5 fold cross validation for this model and the rest of the models.
</p>

<p>
For testing, I choose to do cross validation because it will produce lower variance than a single hold-out set estimator. 5 fold cross validation will reduce variance by averaging over 5 different partitions, so the performance estimate is less sensitive to the partitioning of the data. I plan on doing this for all of the models.
</p>

In [12]:
# Shuffle the data 
X, y = shuffle(X, y)

In [13]:
# Instantiate the linear regressor 
linear_regressor = LinearRegression()

# Perform 5 fold cross validation on the data
linear_scores = cross_validate(linear_regressor, X, y, cv=5, scoring=scoring, n_jobs=-1, verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.0s finished


In [14]:
# Capture the average RMSE and R-squared scores from cross validation
linear_rmse = np.math.sqrt(abs(linear_scores['test_neg_mean_squared_error'].mean()))
linear_r2 = abs(linear_scores['test_r2'].mean())

I will store the results in a new dataframe and then print it.

In [15]:
# Store and print the RMSE and R-squared scores in dataframe
scores_df = pd.DataFrame([{'Model':'Linear Regression', 'RMSE':linear_rmse, 'R-squared':linear_r2}])
print(scores_df.to_string(index=False))

             Model      RMSE  R-squared
 Linear Regression  0.331649    0.50084


<b>RIDGE AND LASSO REGRESSION</b>
<p>
Ridge and lasso regressions are some of the regularization techniques to reduce model complexity and prevent over-fitting which may result from linear regression. They work by penalizing the magnitude of coefficients of variables along with minimizing the error between predicted and actual observations. The key difference is in how they assign penalty to the coefficients. I will run ridge regression first and then lasso. For both of these models, I will also run GridSearchCV to find the best alpha values.
</p>

In [16]:
# Shuffle the data 
X, y = shuffle(X, y)

In [17]:
# Create dictionary of alpha values
param_grid = {'alpha':[0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 500, 1000]}

# Perform 5 fold grid search to get the best alpha value
ridge_gsc = GridSearchCV(
    estimator=Ridge(),
    param_grid=param_grid,
    cv=5, 
    n_jobs=-1,
    verbose=2
)

ridge_grid_result = ridge_gsc.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed:    0.6s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.8s finished


In [18]:
# Get and print the best alpha from the grid search
ridge_best_params = ridge_grid_result.best_params_
print(ridge_best_params)

{'alpha': 5}


In [19]:
# Instantiate ridge regressor with the best alpha
ridge = Ridge(alpha=ridge_best_params['alpha'])

In [20]:
# Perform 5 fold cross validation on the data 
ridge_scores = cross_validate(ridge, X, y, cv=5, scoring=scoring, return_estimator=True)

In [21]:
# Capture the average RMSE and R-squared scores from cross validation
ridge_rmse = np.math.sqrt(abs(ridge_scores['test_neg_mean_squared_error'].mean()))
ridge_r2 = abs(ridge_scores['test_r2'].mean())

I will add the RMSE and r-squared scores of ridge regression to the scores dataframe to keep track of all models in one place.

In [22]:
# Store and print the RMSE and R-squared scores in dataframe
scores_df = scores_df.append(pd.DataFrame([{'Model':'Ridge Regression', 
                                            'RMSE':ridge_rmse, 'R-squared':ridge_r2}]))

print(scores_df.to_string(index=False))

             Model      RMSE  R-squared
 Linear Regression  0.331649    0.50084
  Ridge Regression  0.331505    0.50100


Ridge regression did slightly better than linear regression but not by much.

<b>LASSO REGRESSION</b>
<p>
Unlike ridge regression, lasso regression has the ability to shrink many coefficients to zero if they are not relevant. I will run lasso next.
</p>

In [23]:
# Shuffle the data 
X, y = shuffle(X, y)

In [24]:
# Create dictionary of alpha values
param_grid = {'alpha':[0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 500, 1000]}

# Perform 5 fold grid search to get the best alpha value
lasso_gsc = GridSearchCV(
    estimator=Lasso(),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2
)

lasso_grid_result = lasso_gsc.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  50 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.7s finished


In [25]:
# Get and print the best alpha from the grid search
lasso_best_params = lasso_grid_result.best_params_
print(lasso_best_params)

{'alpha': 0.001}


In [26]:
# Instantiate lasso regressor with the best alpha
lasso = Lasso(alpha=lasso_best_params['alpha'])

In [27]:
# Perform 5 fold cross validation on the data 
lasso_scores = cross_validate(lasso, X, y, cv=5, scoring=scoring, return_estimator=True)

In [28]:
# Capture the average RMSE and R-squared scores from cross validation
lasso_rmse = np.math.sqrt(abs(lasso_scores['test_neg_mean_squared_error'].mean()))
lasso_r2 = abs(lasso_scores['test_r2'].mean())

In [29]:
# Store and print the RMSE and R-squared scores in dataframe
scores_df = scores_df.append(pd.DataFrame([{'Model':'Lasso Regression', 
                                            'RMSE':lasso_rmse, 'R-squared':lasso_r2}]))

print(scores_df.to_string(index=False))

             Model      RMSE  R-squared
 Linear Regression  0.331649    0.50084
  Ridge Regression  0.331505    0.50100
  Lasso Regression  0.331153    0.50205


The scores from ridge and lasso regressions were not very different from the linear regression model. It may mean that the data was not overly complex to begin with and the number of features was not large enough to cause any overfitting. This may be due to the removal of many features during my data wrangling protion of the project.

<b>SUPPORT VECTOR REGRESSION</b>
<p>
Next I will run support vector regression. To optimize the model's accuracy, I will tune it's hyperparameters. There are many available to tune but I will touch on four that are commonly used for this particular model: kernel, C, gamma and epsilon. Finding the best hyperparameters may be computationally expensive, so I will use RandomizedSearchCV first to select a range of values. Then I will feed this into GridSearchCV to find the best ones to use for modeling.
</p>

In [30]:
# Create grid of different hyperparameter values
random_grid = {
    'kernel': ['rbf', 'poly', 'sigmoid'], 
    'C': [0.1, 1, 10, 50, 100, 1000],
    'gamma': [0, 0.01, 0.001, 0.0005, 'auto'],
    'epsilon': [0, 0.01, 0.1, 1, 2]
}

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
svm_random = RandomizedSearchCV(
    estimator=SVR(), 
    param_distributions=random_grid, 
    n_iter=100, 
    cv=3, 
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

In [31]:
# Fit the random search model
svm_grid_result = svm_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 269 out of 300 | elapsed:  3.9min remaining:   26.9s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  8.6min finished


In [344]:
# Get the best hyperparameters from the random search
print(svm_grid_result.best_params_)

{'kernel': 'rbf', 'gamma': 'auto', 'epsilon': 0.1, 'C': 1}


In [345]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'kernel': ['rbf'],
    'gamma': ['auto'],
    'epsilon': [0.05, 0.1, 0.15],
    'C': [0.5, 1, 2]
}

# Instantiate the grid search
svm_gsc = GridSearchCV(
    estimator=SVR(), 
    param_grid=param_grid, 
    cv=3, 
    n_jobs=-1,
    verbose=2
)

# Fit the grid search model
svm_grid_result = svm_gsc.fit(X, y)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  27 | elapsed:   15.1s remaining:   25.6s
[Parallel(n_jobs=-1)]: Done  24 out of  27 | elapsed:   23.6s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   24.5s finished


In [32]:
# Get and print the best hyperparameters from the grid search
svm_best_params = svm_grid_result.best_params_
print(svm_best_params)

{'kernel': 'rbf', 'gamma': 'auto', 'epsilon': 0.1, 'C': 1}


In [33]:
# Instantiate SVM with the best hyperparameters
svm = SVR(
    kernel=svm_best_params['kernel'],
    C=svm_best_params['C'], 
    gamma=svm_best_params['gamma'],
    epsilon=svm_best_params['epsilon'],
)

In [34]:
# Perform 5 fold cross validation on the data 
svm_scores = cross_validate(svm, X, y, cv=5, scoring=scoring)

In [35]:
# Capture the average RMSE and R-squared scores from cross validation
svm_rmse = np.math.sqrt(abs(svm_scores['test_neg_mean_squared_error'].mean()))
svm_r2 = abs(svm_scores['test_r2'].mean())

In [36]:
# Store and print the RMSE and R-squared scores in dataframe
scores_df = scores_df.append(pd.DataFrame([{'Model':'SVM Regression', 
                                            'RMSE':svm_rmse, 'R-squared':svm_r2}]))

print(scores_df.to_string(index=False))

             Model      RMSE  R-squared
 Linear Regression  0.331649   0.500840
  Ridge Regression  0.331505   0.501000
  Lasso Regression  0.331153   0.502050
    SVM Regression  0.321895   0.529561


Support vector regression did slightly better than the first three models.

<b>RANDOM FOREST REGRESSION</b>
<p>
Next I will run random forest regression. To optimize the model's accuracy, I will tune it's hyperparameters just like how I did for support vector regression. There are many hyperparameters available to tune but I will touch on six that are commonly used for this particular model: n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf and bootstrap.
</p>

In [37]:
# Number of trees
n_estimators = [int(x) for x in np.linspace(start=100, stop=4000, num=20)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num=20)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 9, 14, 20]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 7, 11]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create grid of different hyperparameter values
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

print(random_grid)

{'n_estimators': [100, 305, 510, 715, 921, 1126, 1331, 1536, 1742, 1947, 2152, 2357, 2563, 2768, 2973, 3178, 3384, 3589, 3794, 4000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200], 'min_samples_split': [2, 5, 9, 14, 20], 'min_samples_leaf': [1, 2, 4, 7, 11], 'bootstrap': [True, False]}


In [38]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rfr_random = RandomizedSearchCV(
    estimator=RandomForestRegressor(), 
    param_distributions=random_grid, 
    n_iter=100, 
    cv=3, 
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

In [39]:
# Fit the random search model
rfr_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   36.9s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 31.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [40]:
# Get the best hyperparameters from the random search
print(rfr_random.best_params_)

{'n_estimators': 2357, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': False}


In [41]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [60, 70, 80],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [2000, 2250, 2500]
}

# Instantiate the grid search
rfr_gsc = GridSearchCV(
    estimator=RandomForestRegressor(), 
    param_grid=param_grid, 
    cv=3, 
    n_jobs=-1,
    verbose=2
)

In [42]:
# Fit the grid search model
rfr_grid_result = rfr_gsc.fit(X, y)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  9.4min finished


In [43]:
# Get and print the best hyperparameters from the grid search
rfr_best_params = rfr_grid_result.best_params_
print(rfr_best_params)

{'bootstrap': False, 'max_depth': 70, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 2250}


In [44]:
# Instantiate random forest regressor with the best hyperparameters
rfr = RandomForestRegressor(
    bootstrap=rfr_best_params['bootstrap'],
    max_depth=rfr_best_params['max_depth'], 
    max_features=rfr_best_params['max_features'],
    min_samples_leaf=rfr_best_params['min_samples_leaf'],
    min_samples_split=rfr_best_params['min_samples_split'],
    n_estimators=rfr_best_params['n_estimators']
)

In [45]:
# Perform 5 cross validation on the data
rfr_scores = cross_validate(rfr, X, y, cv=5, scoring=scoring, n_jobs=-1, verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   28.2s finished


In [46]:
# Capture the average RMSE and R-squared scores from cross validation
rfr_rmse = np.math.sqrt(abs(rfr_scores['test_neg_mean_squared_error'].mean()))
rfr_r2 = abs(rfr_scores['test_r2'].mean())

In [47]:
# Store and print the RMSE and R-squared scores in dataframe
scores_df = scores_df.append(pd.DataFrame([{'Model':'Random Forest Regression', 
                                            'RMSE':rfr_rmse, 'R-squared':rfr_r2}]))

print(scores_df.to_string(index=False))

                    Model      RMSE  R-squared
        Linear Regression  0.331649   0.500840
         Ridge Regression  0.331505   0.501000
         Lasso Regression  0.331153   0.502050
           SVM Regression  0.321895   0.529561
 Random Forest Regression  0.267853   0.674272


Random forest regression did significantly better than the first four models. 

<b>GRADIENT BOOSTING</b>
<p>
Next I will run gradient boosting. To optimize the model's accuracy, I will tune it's hyperparameters just like how I did for the previous two. There are many hyperparameters available to tune but I will touch on six that are commonly used for this particular model: learning_rate, n_estimators, max_depth, min_samples_split, min_samples_leaf and max_features.
</p>

In [48]:
# Create grid of different hyperparameter values
random_grid = {
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
    'n_estimators': n_estimators, 
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'max_features': max_features
}

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
gbr_random = RandomizedSearchCV(
    estimator=GradientBoostingRegressor(), 
    param_distributions=random_grid, 
    n_iter=100, 
    cv=3, 
    verbose=2, 
    random_state=42, 
    n_jobs=-1
)

In [49]:
# Fit the random search model
gbr_grid_result = gbr_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed: 67.3min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 177.6min finished


In [50]:
# Get the best hyperparameters from the random search
print(gbr_grid_result.best_params_)

{'n_estimators': 3589, 'min_samples_split': 20, 'min_samples_leaf': 7, 'max_features': 'sqrt', 'max_depth': 40, 'learning_rate': 0.001}


In [51]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'learning_rate': [0.001],
    'max_depth': [30, 40, 50],
    'min_samples_split': [15, 20, 25],
    'n_estimators': [3000, 3500, 4000],
    'min_samples_leaf': [6, 7, 8],
    'max_features': ['sqrt']
}

# Instantiate the grid search
gbr_gsc = GridSearchCV(
    estimator=GradientBoostingRegressor(), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs=-1,
    verbose=2
)

In [52]:
# Fit the grid search model
gbr_grid_result = gbr_gsc.fit(X, y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed: 44.2min
[Parallel(n_jobs=-1)]: Done 333 tasks      | elapsed: 140.0min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed: 208.5min finished


In [53]:
# Get and print the best hyperparameters from the grid search
gbr_best_params = gbr_grid_result.best_params_
print(gbr_best_params)

{'learning_rate': 0.001, 'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 6, 'min_samples_split': 15, 'n_estimators': 4000}


In [54]:
# Instantiate gradient boosting regressor with the best hyperparameters
gbr = GradientBoostingRegressor(
    learning_rate=gbr_best_params['learning_rate'],
    max_depth=gbr_best_params['max_depth'], 
    min_samples_split=gbr_best_params['min_samples_split'],
    n_estimators=gbr_best_params['n_estimators']
)

In [55]:
# Perform 5 cross validation on the data
gbr_scores = cross_validate(gbr, X, y, cv=5, scoring=scoring, n_jobs=-1, verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 29.7min finished


In [56]:
# Capture the average RMSE and R-squared scores from cross validation
gbr_rmse = np.math.sqrt(abs(gbr_scores['test_neg_mean_squared_error'].mean()))
gbr_r2 = abs(gbr_scores['test_r2'].mean())

In [57]:
# Store and print the RMSE and R-squared scores in dataframe
scores_df = scores_df.append(pd.DataFrame([{'Model':'Gradient Boosting Regression', 
                                            'RMSE':gbr_rmse, 'R-squared':gbr_r2}]))

print(scores_df.to_string(index=False))

                        Model      RMSE  R-squared
            Linear Regression  0.331649   0.500840
             Ridge Regression  0.331505   0.501000
             Lasso Regression  0.331153   0.502050
               SVM Regression  0.321895   0.529561
     Random Forest Regression  0.267853   0.674272
 Gradient Boosting Regression  0.290646   0.616401


Random forest regression was the best performing model at 0.67 r-squared and 0.27 RMSE. Gradient boosting regression did slightly worse than it. I would like to see a visual comparison of the RMSE and r-squared scores for all of the models.

In [69]:
# Create graph for RMSE scores
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=scores_df['Model'],
    y=scores_df['RMSE'],
    marker=dict(color="orange", size=12),
    mode="markers",
))

fig.update_layout(title="RMSE Scores",
                  yaxis_title="RMSE")

fig.show()

In [72]:
# Create graph for r-squared scores
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=scores_df['Model'],
    y=scores_df['R-squared'],
    marker=dict(color="orange", size=12),
    mode="markers",
))

fig.update_layout(title="R-squared Scores",
                  yaxis_title="r-squared")

fig.show()

I can see from the graphs above that random forest regression had the highest r-squared and also had the lowest RMSE score. Gradient boosting regression wasn't very far behind for both r-squared and RMSE. Linear, ridge and lasso regressions were the worst performing models with relatively similar scores.